In [1]:
import os

In [5]:
%pwd

'd:\\text_Summerizer\\text_summarizer'

In [4]:
os.chdir("../")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir:Path
    data_path: Path
    model_path: Path
    metric_file_name: Path

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories


In [ ]:
class ConfriguationManager:
    def __init__(self,
                config_file_path=CONFIG_FILE_PATH,
                params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir =config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name
        )
    